In [4]:
import import_ipynb
from preprocess import *

import numpy as np
import pandas as pd
import seaborn as sns
import heapq

In [ ]:
import heapq
import math
import numpy as np

def normalize_surface(surface):
    """Normalize the energy surface to values between 0 and 1."""
    E_min, E_max = np.min(surface), np.max(surface)
    if E_max == E_min:
        return np.zeros_like(surface, dtype=float)
    return (surface - E_min) / (E_max - E_min)

def dijkstra_waed(surface, start, goal, uphill_weight=1.0, downhill_weight=0.5):
    """
    Dijkstra's algorithm using WAED with metric outputs.

    Returns:
    - path: list of (x, y) nodes
    - metrics: dict with path metrics
    """
    norm_surface = normalize_surface(surface)
    rows, cols = norm_surface.shape
    directions = [(-1,0), (1,0), (0,-1), (0,1), (-1,-1), (-1,1), (1,-1), (1,1)]

    def get_neighbors(x, y):
        return [((x + dx) % rows, (y + dy) % cols) for dx, dy in directions]

    def waed_cost(current, neighbor):
        x1, y1 = current
        x2, y2 = neighbor
        Ei, Ej = norm_surface[x1, y1], norm_surface[x2, y2]
        delta = Ej - Ei
        return uphill_weight * delta if delta >= 0 else downhill_weight * abs(delta)

    # Dijkstra
    open_list = []
    heapq.heappush(open_list, (0, start))
    g_score = {start: 0}
    came_from = {}

    while open_list:
        _, current = heapq.heappop(open_list)
        if current == goal:
            # Reconstruct path
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            path.reverse()

            # Compute metrics
            energy_vals = np.array([norm_surface[x, y] for x, y in path])
            steps = np.abs(np.diff(energy_vals))
            first_diffs = np.diff(energy_vals)
            smoothness = float(np.sum(np.abs(np.diff(first_diffs)))) if len(path) >= 3 else 0.0

            metrics = {
                'path_exists': True,
                'path_length': len(path),
                'height': float(np.max(energy_vals) - np.min(energy_vals)),
                'total_cost': float(np.sum(steps)),
                'smoothness': smoothness,
                'max_step': float(np.max(steps)) if len(steps) > 0 else 0.0
            }

            return path, metrics

        for neighbor in get_neighbors(*current):
            tentative_g = g_score[current] + waed_cost(current, neighbor)
            if neighbor not in g_score or tentative_g < g_score[neighbor]:
                g_score[neighbor] = tentative_g
                came_from[neighbor] = current
                heapq.heappush(open_list, (tentative_g, neighbor))

    # If no path found
    return None, {
        'path_exists': False,
        'path_length': 0,
        'height': None,
        'total_cost': None,
        'smoothness': None,
        'max_step': None
    }
